# Analyze Product Sentiment

In [1]:
import turicreate

# Read product review data

In [2]:
products = turicreate.SFrame('amazon_baby.sframe')

# Explore data

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [4]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [5]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [6]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0


In [7]:
len(giraffe_reviews)

785

In [8]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [9]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [10]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1.0,'stripping': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'highly': 1.0, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ..."


In [11]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']
cnt = 0
def awesome_count(obj):
    if(selected_words[cnt] in obj):
        return obj[selected_words[cnt]]
    else:
        return 0
for i in selected_words:
    products[i] = products['word_count'].apply(awesome_count)
    cnt+=1


for i in selected_words:
    print((products[i].sum(),i))



        
        
    

(4075.0, 'awesome')
(59536.0, 'great')
(1765.0, 'fantastic')
(2726.0, 'amazing')
(43867.0, 'love')
(1245, 'horrible')
(4950.0, 'bad')
(1282, 'terrible')
(753, 'awful')
(461, 'wow')
(1285, 'hate')


# Define what is positive and negative sentiment

In [12]:
products['rating'].show()

Materializing SArray

In [13]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [14]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [15]:
products

name,review,rating,word_count,awesome,great
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'highly': 1.0, ...",0.0,0.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",0.0,1.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",0.0,1.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",0.0,0.0


In [16]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [17]:
train_data,test_data = products.random_split(.8,seed=0)

In [18]:
selected_words_sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 2         | 11       | 2.669978  | 2.605519     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 2.980285     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 3.559093     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 5.569436     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [19]:
x = selected_words_sentiment_model.coefficients
x = x.sort('value')
for i in x:
    print(i['name'],i['value'],i['class'],i['stderr'])

horrible -2.251335236759102 1 0.08020249388788425
terrible -2.2236614360851346 1 0.07731736203785744
awful -2.0529082040313598 1 0.10099735435259247
hate -1.3484407222463202 1 0.07715698604297323
bad -0.9914778800650664 1 0.03848428664699063
wow -0.00953823606768843 1 0.16046411224711649
great 0.8630655001196439 1 0.018955052444376685
fantastic 0.88580475688142 1 0.11167591293399651
amazing 1.100093311366018 1 0.0995477626046598
awesome 1.133534666034134 1 0.08399643983187562
(intercept) 1.3365913848877726 1 0.008929969787655916
love 1.359268866922504 1 0.02806830015209953


In [20]:
model_1 = selected_words_sentiment_model.evaluate(test_data)
model_2 = sentiment_model.evaluate(test_data)
print(model_1)
print("::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")
print(model_2)


{'accuracy': 0.8463848186404036, 'auc': 0.6936022046674926, 'confusion_matrix': Columns:
	target_label	int
	predicted_label	int
	count	int

Rows: 4

Data:
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      1       |        0        |  159  |
|      0       |        0        |  371  |
|      0       |        1        |  4957 |
|      1       |        1        | 27817 |
+--------------+-----------------+-------+
[4 rows x 3 columns]
, 'f1_score': 0.9157860082304526, 'log_loss': 0.39622654670876317, 'precision': 0.8487520595594068, 'recall': 0.9943165570488991, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 100001

Data:
+-----------+--------------------+-----+-------+------+
| threshold |        fpr         | tpr |   p   |  n   |
+-----------+--------------------+-----+-------+------+
|    0.0    |        1.0         | 1.0 | 27976 | 5328 |
|   1e-05   |        1.0      

# Apply the sentiment classifier to better understand the Giraffe reviews

In [21]:
products['predicted_sentiment_selected_words'] = selected_words_sentiment_model.predict(products, output_type = 'probability')
products['predicted_sentiment_all_words'] = sentiment_model.predict(products, output_type = 'probability')

In [22]:
products

name,review,rating,word_count,awesome,great
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'highly': 1.0, ...",0.0,0.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",0.0,0.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",0.0,1.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",0.0,1.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",0.0,0.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",0.0,0.0


In [23]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']


In [32]:
print(diaper_champ_reviews)
result_1 = diaper_champ_reviews.sort('predicted_sentiment_selected_words')
print("Selected Words")
# print(result_1.head())
# print(result_1.tail())
for i in selected_words:
    cnt=0
    for j in result_1[-1]['word_count'].keys():
        if(i==j):
            cnt+=1
    print(i,cnt)
result_2 = diaper_champ_reviews.sort('predicted_sentiment_all_words')
print("All Words")
# print(result_2[-1]['word_count'])
# print(result_2.head())
# print(result_2.tail()) 
for i in selected_words:
    cnt=0
    for j in result_2[-1]['word_count'].keys():
        if(i==j):
            cnt+=1
    print(i,cnt)

+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Ok - newsflash.  Diapers a... |  4.0   |
| Baby Trend Diaper Champ | My husband and I selected ... |  1.0   |
| Baby Trend Diaper Champ | Excellent diaper disposal ... |  5.0   |
| Baby Trend Diaper Champ | We love our diaper champ. ... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I waited to review this un... |  4.0   |
| Baby Trend Diaper Champ | I have had a diaper genie ... |  1.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | I am so glad I got the Dia... |  5.0   |
| Baby Trend Diaper Champ | We had 2 diaper Genie's bo... |  4.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-

# Sort the Giraffe reviews according to predicted sentiment

In [72]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [73]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,"Great feel, great squeek,great quality, great ...",5.0,"{'forever': 1.0,'saving': 1.0, 'll': ...",1
Vulli Sophie the GiraffeTeether ...,Sophie is one of mydaughter's favorite t ...,5.0,"{'love': 3.0, 'she': 1.0,'wonderful': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,Love it! Love it! Loveit! The best teether I ...,5.0,"{'this': 1.0, 'before':1.0, 'others': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,We love Sophie at ourhouse... she is a great ...,5.0,"{'so': 1.0, 'too': 1.0,'which': 1.0, 'natural': ...",1
Vulli Sophie the GiraffeTeether ...,Had one for my first twochildren but it had seen ...,5.0,"{'parents': 1.0, 'these':1.0, 'great': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,My 5mo daughter iscompletely in love with ...,5.0,"{'baby': 1.0, 'gift':1.0, 'slowly': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,All my grandbabies lovethis toy. It smells g ...,5.0,"{'because': 1.0, 'on':1.0, 'tasts': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,"Being a childphotographer, I saw lots ...",4.0,"{'though': 1.0, 'will':1.0, 'expect': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,We just got our adorableSophie this week and ...,5.0,"{'paint': 1.0, 'rubber':1.0, 'from': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,This is the first reviewI've written for ...,4.0,"{'worth': 1.0, 'few':1.0, 'little': 1.0, ...",1


In [74]:
giraffe_reviews.tail()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,I had a Sophie I hadforgotten I bought. ...,1.0,"{'ones': 1.0, 'given':1.0, 'haven': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,An excellent toy forteething babies - it is ...,4.0,"{'lovely': 1.0, 'extra':1.0, 'willing': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,When I received this thepaint was peeling off in ...,1.0,"{'risk': 1.0, 'but': 1.0,'help': 1.0, 'terrible': ...",0
Vulli Sophie the GiraffeTeether ...,I was very pleased withour Sophie the Giraffe. ...,4.0,"{'box': 1.0, 'of': 1.0,'out': 1.0, 'you': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,It's really terrible.When my 4 months daug ...,1.0,"{'away': 1.0, 'right':1.0, 'my': 1.0, 'the': ...",0
Vulli Sophie the GiraffeTeether ...,I thought it was reallygood toy and bought it. ...,1.0,"{'crap': 1.0, 'your':1.0, 'poison': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,"With only a few daysworth of minimal play, I ...",1.0,"{'how': 1.0,'especially': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,I got one of these as ashowe gift that my baby ...,1.0,"{'typical': 1.0, 'paint':1.0, 'came': 1.0, 'mi ...",0
Vulli Sophie the GiraffeTeether ...,I received two of theseat my baby shower. I ...,2.0,"{'parent': 1.0, 'this':1.0, 'never': 1.0, ...",0
Vulli Sophie the GiraffeTeether ...,"When I first heard aboutthis teether, I thought ...",5.0,"{'term': 1.0, 'damage':1.0, 'bath': 1.0, 'ta ...",1


## Show the most positive reviews

In [75]:
giraffe_reviews[0]['review']

"Great feel, great squeek, great quality, great story...Sophie is just great all around. My little man loves her...even though in public I do feel a little odd asking my son &#34;here honey baby, do you want your Sophie doll&#34;? Hubs wanted to rename her to a boy name....but that would ruin Sophie's legacy. My son played with her up to about a year old..I'll be saving her forever in my keepsake box."

In [76]:
giraffe_reviews[1]['review']

"Sophie is one of my daughter's favorite toys, and is wonderful as she begins teething.  Love love love Sophie!"

# Most negative reivews

In [77]:
giraffe_reviews[-1]['review']

'When I first heard about this teether, I thought it was just a stupid expensive yuppie thing that is overpriced and appeals only to people so much money they don\'t know what to do with it.  I was dead wrong.  My daughter "tried" her cousin\'s Sophie when she was 7 months old and in a horrible bout of teething, and she didn\'t want to give it back.  I went out and purchased a Sophie for her the very next day.  This is the only teething toy that ever gave her any relief during teething, and she had a terrible time cutting teeth.  The quality of the toy reflects the price.  She dropped her Sophie at the zoo without me noticing one day, and I had to buy her another one. It\'s that good.  I think Sophie is a perfect baby shower gift, as it is tough for us new parents to justify spending so much money on a teether.  Buy it for a pregnant mom! Or if you can spare the cash, definitely go ahead and buy it for your baby.  I don\'t think you\'ll regret it! Note of caution: don\'t let baby take 

In [78]:
giraffe_reviews[-2]['review']

'I received two of these at my baby shower. I thought they were cute and then I opened one and gave it to my baby. IT SQUEAKS!!!!! It makes a high-pitched, dog-toy squeak that is obnoxious. That being said, the baby loves chewing on it and it is easy for her to hold. But that noise - it is awful. It is loud and draws attention. I will not take it with us to restaurants or even in the car.  It is so bad I have considered &#34;losing&#34; Sophie. I would never give this to another parent.'